In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
os.chdir("drive/My Drive/face")

In [ ]:
import pandas as pd
import numpy as np

Reading csv file with training data.

In [ ]:
df = pd.read_csv("training.csv")

In [ ]:
df.values.shape

In [ ]:
images = df.Image.values

Converting images to numpy.

In [ ]:
images = np.zeros((df.values.shape[0], 96, 96))
for i in range(df.values.shape[0]):
    images[i] = np.fromstring(df.Image.values[i], 
                              sep = " ").reshape(96,96)

In [ ]:
images.shape

In [ ]:
coords = df.values[:, :30].astype("float")

In [ ]:
coords.shape

Deleting images and coord vectors with nan values.

In [ ]:
isNanMatrix = np.isnan(coords)

In [ ]:
rowsWithNan = isNanMatrix.any(axis=1)

In [ ]:
coordsNew = coords[~rowsWithNan]
imagesNew = images[~rowsWithNan]

In [ ]:
coordsNew.shape

In [ ]:
imagesNew.shape

Rounding coords to create pixel tensor containing 1-s at positions of special points and zeros elsewhere.

In [ ]:
coordsNewRounded = np.round(coordsNew).astype(int) - 1

In [ ]:
coordsNewReshaped = coordsNewRounded.reshape\
(imagesNew.shape[0], 15, 2)

In [ ]:
pixelTensor = np.zeros((imagesNew.shape[0], 96, 96, 15), 
                       dtype = float)
for image in range(imagesNew.shape[0]):
    for feature in range(15):
        pixelTensor[image, 
                    coordsNewReshaped[image, feature, 1],
                    coordsNewReshaped[image, feature, 0],
                    feature] = 1.0

Checking values of tensor.

In [ ]:
np.unravel_index(pixelTensor[0,:,:,0].argmax(), 
                 (96,96))

In [ ]:
pixelTensor[0, 38, 65, 0]

In [ ]:
coordsNewReshaped[0][0]

In [ ]:
imagesNew.shape

In [ ]:
pixelTensor.shape

In [ ]:
imagesNew = imagesNew.reshape\
(imagesNew.shape + (1,))/255

In [ ]:
imagesNew.shape

Splitting into train and test set

In [ ]:
images_train = imagesNew[:1500, :, :, :]
images_test = imagesNew[1500:, :, :, :]
pixel_train = pixelTensor[:1500, :, :, :]
pixel_test = pixelTensor[1500:, :, :, :]

Building the model (this type of network is not working well). We use layerwise softmax at the last layer 
(last tensor has depth 15 that corresponds to 15 points of interest on the image)

In [ ]:
import keras

In [ ]:
def softMaxAxis1(x):
    return keras.activations.softmax(x,axis=-1)

In [ ]:
from keras.layers import Conv2D, Dropout

In [ ]:
model = keras.models.Sequential()
model.add(Conv2D(16, (3,3), padding="same", 
        activation="relu", input_shape=(96,96,1)))
model.add(Dropout(0.2))
model.add(Conv2D(24, (3,3), padding="same", 
        activation="relu"))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), padding="same", 
        activation="relu"))
model.add(Dropout(0.2))
model.add(Conv2D(40, (3,3), padding="same", 
        activation="relu"))
model.add(Dropout(0.2))
model.add(Conv2D(15, (3,3), padding="same",
        activation=softMaxAxis1))
#model.add(keras.activations.softmax(axis = -1))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), 
        loss = keras.losses.categorical_crossentropy)

In [ ]:
model.fit(images_train, pixel_train, 
          batch_size=32, epochs=10)

In [ ]:
score = model.evaluate(images_test, pixel_test)

In [ ]:
print(score)

Saving the trained model to google drive

In [ ]:
model.save("model_face")

In [ ]:
model.save_weights("face_weights")